In [1]:
import numpy as np
import pandas as pd
import os

import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm_notebook
from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVR, SVR
from sklearn.metrics import mean_absolute_error


from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import lightgbm as lgb
import time
import datetime
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold
from sklearn.metrics import mean_absolute_error
import gc
import seaborn as sns
import warnings

from scipy.signal import hilbert
from scipy.signal import hann
from scipy.signal import convolve
from scipy import stats
from sklearn.kernel_ridge import KernelRidge
from tsfresh.feature_extraction import feature_calculators
import copy

In [2]:
train = pd.read_csv("../input/train.csv", dtype={'acoustic_data': np.int16, 'time_to_failure': np.float32})

In [3]:

# Display time_to_failure with more units of precision
pd.options.display.precision = 15

# Create a training file with simple derived features
rows = 150_000
segments = int(np.floor(train.shape[0] / rows))

X_tr = pd.DataFrame(index=range(segments), dtype=np.float64)

y_train = pd.DataFrame(index=range(segments), dtype=np.float64, columns=['time_to_failure'])

total_mean = train['acoustic_data'].mean()
total_std = train['acoustic_data'].std()
total_max = train['acoustic_data'].max()
total_min = train['acoustic_data'].min()
total_sum = train['acoustic_data'].sum()
total_abs_sum = np.abs(train['acoustic_data']).sum()

for segment in tqdm_notebook(range(segments)):
    seg = train.iloc[segment*rows:segment*rows+rows]
    x = pd.Series(seg['acoustic_data'].values)
    y = seg['time_to_failure'].values[-1]
    y_train.loc[segment, 'time_to_failure'] = y
    
    zc = np.fft.fft(x)
    realFFT = np.real(zc)
    imagFFT = np.imag(zc)
    X_tr.loc[segment, 'Rmean'] = realFFT.mean()
    X_tr.loc[segment, 'Rstd'] = realFFT.std()
    X_tr.loc[segment, 'Rmax'] = realFFT.max()
    X_tr.loc[segment, 'Rmin'] = realFFT.min()
    X_tr.loc[segment, 'Imean'] = imagFFT.mean()
    X_tr.loc[segment, 'Istd'] = imagFFT.std()
    X_tr.loc[segment, 'Imax'] = imagFFT.max()
    X_tr.loc[segment, 'Imin'] = imagFFT.min()
    
    
    X_tr.loc[segment, 'mean'] = x.mean()
    X_tr.loc[segment, 'std'] = x.std()
    X_tr.loc[segment, 'max'] = x.max()
    X_tr.loc[segment, 'min'] = x.min()
    X_tr.loc[segment, 'kurt'] = x.kurtosis()
    X_tr.loc[segment, 'num_peaks_10'] = feature_calculators.number_peaks(x, 10)
    X_tr.loc[segment, 'autocorrelation_5'] = feature_calculators.autocorrelation(x, 5)
    
    for windows in [10, 100, 1000]:
        x_roll_std = x.rolling(windows).std().dropna().values
        x_roll_mean = x.rolling(windows).mean().dropna().values
        
        X_tr.loc[segment, 'max_roll_mean_' + str(windows)] = x_roll_mean.max()
        X_tr.loc[segment, 'min_roll_mean_' + str(windows)] = x_roll_mean.min()
        X_tr.loc[segment, 'q01_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.01) #Als je niet weet wat dit doet vraag het aan Pepijn
        X_tr.loc[segment, 'q05_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.05)
        X_tr.loc[segment, 'q95_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.95)
        
        X_tr.loc[segment, 'q01_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.01)
        X_tr.loc[segment, 'q05_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.05)
        X_tr.loc[segment, 'q95_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.95)
        X_tr.loc[segment, 'min_roll_std_' + str(windows)] = x_roll_std.min()
        X_tr.loc[segment, 'ave_roll_mean' + str(windows)] = x_roll_mean.mean()

In [4]:
means_dict = {}
for col in X_tr.columns:
    if X_tr[col].isnull().any():
        print(col)
        mean_value = X_tr.loc[X_tr[col] != -np.inf, col].mean()
        X_tr.loc[X_tr[col] == -np.inf, col] = mean_value
        X_tr[col] = X_tr[col].fillna(mean_value)
        means_dict[col] = mean_value

scaler = StandardScaler()
scaler.fit(X_tr)
X_train_scaled = pd.DataFrame(scaler.transform(X_tr), columns=X_tr.columns)

In [5]:
submission = pd.read_csv('../input/sample_submission.csv', index_col='seg_id')
X_test = pd.DataFrame(columns=X_tr.columns, dtype=np.float64, index=submission.index)

for i, seg_id in enumerate(tqdm_notebook(X_test.index)):
    seg = pd.read_csv('../input/test/' + seg_id + '.csv')
    
    x = pd.Series(seg['acoustic_data'].values)
    zc = np.fft.fft(x)
    realFFT = np.real(zc)
    imagFFT = np.imag(zc)
    
    X_test.loc[seg_id, 'Rmean'] = realFFT.mean()
    X_test.loc[seg_id, 'Rstd'] = realFFT.std()
    X_test.loc[seg_id, 'Rmax'] = realFFT.max()
    X_test.loc[seg_id, 'Rmin'] = realFFT.min()
    X_test.loc[seg_id, 'Imean'] = imagFFT.mean()
    X_test.loc[seg_id, 'Istd'] = imagFFT.std()
    X_test.loc[seg_id, 'Imax'] = imagFFT.max()
    X_test.loc[seg_id, 'Imin'] = imagFFT.min()
    
    X_test.loc[seg_id, 'mean'] = x.mean()
    X_test.loc[seg_id, 'std'] = x.std()
    X_test.loc[seg_id, 'max'] = x.max()
    X_test.loc[seg_id, 'min'] = x.min()  

    X_test.loc[seg_id, 'kurt'] = x.kurtosis()

    X_test.loc[seg_id, 'num_peaks_10'] = feature_calculators.number_peaks(x, 10)
    X_test.loc[seg_id, 'autocorrelation_5'] = feature_calculators.autocorrelation(x, 5)
    
    for windows in [10, 100, 1000]:
        x_roll_std = x.rolling(windows).std().dropna().values
        x_roll_mean = x.rolling(windows).mean().dropna().values
                
        X_test.loc[seg_id, 'max_roll_mean_' + str(windows)] = x_roll_mean.max()
        X_test.loc[seg_id, 'min_roll_mean_' + str(windows)] = x_roll_mean.min()
        X_test.loc[seg_id, 'q01_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.01)
        X_test.loc[seg_id, 'q05_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.05)
        X_test.loc[seg_id, 'q95_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.95)
        
        X_test.loc[seg_id, 'q01_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.01)
        X_test.loc[seg_id, 'q05_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.05)
        X_test.loc[seg_id, 'q95_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.95)
        X_test.loc[seg_id, 'min_roll_std_' + str(windows)] = x_roll_std.min()
        X_test.loc[seg_id, 'ave_roll_mean' + str(windows)] = x_roll_mean.mean()
        
for col in X_test.columns:
    if X_test[col].isnull().any():
        X_test.loc[X_test[col] == -np.inf, col] = means_dict[col]
        X_test[col] = X_test[col].fillna(means_dict[col])
        
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)


In [6]:
from gplearn.genetic import SymbolicRegressor
from gplearn.functions import make_function

def tanh(x):
    return np.tanh(x);
def sinh(x):
    return np.sinh(x);

gp_tanh = make_function(tanh,"tanh",1)
gp_sinh = make_function(sinh,"sinh",1)
f_set = ['add', 'sub', 'mul', 'div', 'inv', 'abs', 'neg', 'max', 'min','sqrt','log', gp_tanh, gp_sinh]

In [7]:
est_gp = SymbolicRegressor(population_size=2000,
                               tournament_size=50,
                               generations=40, stopping_criteria=0.0,
                               p_crossover=0.7, p_subtree_mutation=0.05, p_hoist_mutation=0.05, p_point_mutation=0.05,
                               max_samples=1.0, verbose=1,
                               function_set = (gp_tanh,'sqrt', 'add', 'sub', 'mul', 'div','log','sin','cos','neg','max','min','abs'),
                               metric = 'mean absolute error', warm_start=True,
                               n_jobs = -1, parsimony_coefficient=0.00001, random_state=11)

In [8]:
predictions = np.zeros(len(X_test_scaled))

n_fold = 5
folds = KFold(n_splits=n_fold, shuffle=False, random_state=42)

fold_importance_df = pd.DataFrame()
fold_importance_df["Feature"] = X_train_scaled.columns


In [9]:
GENS = 75
MAE_THRESH = 2.5
MAX_NO_IMPROVE = 50

maes = []
gens = []

for fold_, (trn_idx, val_idx) in enumerate(folds.split(X_train_scaled, y_train.values.flatten())):
    X_tr, X_val = X_train_scaled.iloc[trn_idx], X_train_scaled.iloc[val_idx]
    y_tr, y_val = y_train['time_to_failure'].values[trn_idx].ravel(), y_train['time_to_failure'].values[val_idx].ravel()
    best = 1e10
    count = 1
    imp_count = 0
    best_mdl = None
    best_iter = 0
    gp = SymbolicRegressor(population_size=2000,
                               tournament_size=30,
                               generations=count, stopping_criteria=0.0,
                               p_crossover=0.7, p_subtree_mutation=0.1, p_hoist_mutation=0.1, p_point_mutation=0.1,
                               max_samples=1.0, verbose=1,
                               function_set = (gp_tanh,'sqrt', 'add', 'sub', 'mul', 'div','log','sin','cos','neg','max','min','abs'),
                               metric = 'mean absolute error', warm_start=True,
                               n_jobs = -1, parsimony_coefficient=0.00001, random_state=11)

    for run in range(GENS):
        mdl = gp.fit(X_tr, y_tr)
        pred = gp.predict(X_val)
        mae = np.sqrt(mean_absolute_error(y_val, pred))

        if mae < best and imp_count < MAX_NO_IMPROVE:
            best = mae
            count += 1
            gp.set_params(generations=count, warm_start=True)
            imp_count = 0
            best_iter = run
            if mae < MAE_THRESH:
                best_mdl = copy.deepcopy(mdl)
        elif imp_count < MAX_NO_IMPROVE:
            count += 1
            gp.set_params(generations=count, warm_start=True)
            imp_count += 1
        else:
            break

        print('GP MAE: %.4f, Run: %d, Best Run: %d, Fold: %d' % (mae, run, best_iter, fold_))

    maes.append(best)
    gens.append(run)
      
    print('Finish - GP MAE: %.4f, Run: %d, Best Run: %d' % (mae, run, best_iter))
          
    preds = best_mdl.predict(X_test_scaled)
    print(preds[0:12])
    predictions += preds / folds.n_splits

try:
    print(maes)
    print(np.mean(maes))
    print(gens)
except:
    print('oops')
submission.time_to_failure = predictions
submission.to_csv('submission_gplearn_AF0_1.csv')

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0     9.16          7.09622       14          3.37427              N/A      0.00s
GP MAE: 1.8476, Run: 0, Best Run: 0, Fold: 0
    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   1    14.45          5.34254       23          3.24902              N/A      0.00s
GP MAE: 1.8212, Run: 1, Best Run: 1, Fold: 0
    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   2 

In [10]:

#Code above with kfolds gets better result Than this simple one
"""
est_gp.fit(X_train_scaled , y_train.values.flatten())

y_gp = est_gp.predict(X_train_scaled)
gpLearn_MAE = mean_absolute_error(y_tr.values.flatten(), y_gp)
print("gpGpiLearn MAE:", gpLearn_MAE)
"""


'\nest_gp.fit(X_train_scaled , y_train.values.flatten())\n\ny_gp = est_gp.predict(X_train_scaled)\ngpLearn_MAE = mean_absolute_error(y_tr.values.flatten(), y_gp)\nprint("gpGpiLearn MAE:", gpLearn_MAE)\n'

In [11]:
"""

# This can be used to create a download link for the submission file

from IPython.display import HTML
import base64

def create_download_link(df, title = "Download CSV file", filename = "data.csv"):  
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

create_download_link(submission)
"""

'\n\n# This can be used to create a download link for the submission file\n\nfrom IPython.display import HTML\nimport base64\n\ndef create_download_link(df, title = "Download CSV file", filename = "data.csv"):  \n    csv = df.to_csv()\n    b64 = base64.b64encode(csv.encode())\n    payload = b64.decode()\n    html = \'<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>\'\n    html = html.format(payload=payload,title=title,filename=filename)\n    return HTML(html)\n\ncreate_download_link(submission)\n'